In [62]:
# from dateparser import parse
# from dateparser.search import search_dates
# time = search_dates('I need a desk for tomorrow from 2pm for 2 hours')
# print(time)
# # time = parse('14 Ramadan')
# # print(time)

In [ ]:
# import pandas as pd
# df = pd.read_csv(r"C:\Danyal\Work\FAST\Semester 7\Final Year Project - I\Dummy Project\NAaaS\Data_Generator\2022\2022-07-16\international.csv")
# df.head()

In [ ]:
# import json
# from sutime import SUTime
# test_case = 'I need a desk'
# sutime = SUTime(mark_time_ranges=True, include_range=True)
# # print(json.dumps(sutime.parse(test_case), sort_keys=True, indent=4))
# timeData = dict()

# for data in df.itertuples():
#     pass
#     # print(data[4])
#     # print(json.dumps(sutime.parse(data[2]), sort_keys=True))
#     timeData[data[0]] = dict()
#     timeData[data[0]]["Header"] = json.dumps(sutime.parse(data[2]), sort_keys=True)
#     timeData[data[0]]['Summary'] = json.dumps(sutime.parse(data[3]), sort_keys=True)
#     timeData[data[0]]['Details'] = json.dumps(sutime.parse(data[4]), sort_keys=True)



In [2]:
# len(timeData)
import glob
import pandas as pd
import pathlib
li = []
for filename in glob.iglob(r'..\Data_Generator\2022\**\*.csv', recursive=True):
    path = pathlib.PurePath(filename)
    # print(path.parent.name)
    df = pd.read_csv(filename, index_col=None, header=0, dtype="string")
    df['Creation_Date'] = path.parent.name
    li.append(df)
df = pd.concat(li, axis=0, ignore_index=True)

# for i in range(len(timeData)):
#     print(timeData[i]['Header'])
df.head()

,Unnamed: 0,Header,Summary,Detail,Creation_Date
0,0,SITUATIONER: HOW CAN PAKISTAN WIN 5G ‘WAR’ AGA...,India has used its local laws to successfully ...,WHILE India recently blocked social media acco...,2022-07-12
1,1,80pc of Pakistanis’ wealth comprises residenti...,Asset accumulation is slower early in the life...,ISLAMABAD: A new World Bank study says Pakista...,2022-07-12
2,2,"IMF, Pakistan inch closer to revival of loan p...",Development comes as Islamabad has taken steps...,WASHINGTON: The International Monetary Fund (I...,2022-07-12
3,3,"Blinken, Wang hold talks on trade, Ukraine war",Biden and Xi are expected to speak in coming w...,NUSA DUA: US Secretary of State Antony Blinken...,2022-07-12
4,4,10 dams breached as rains ravage Balochistan,"CM orders probe into dams’ ""faulty"" constructi...",QUETTA: Authorities in Balochistan have issued...,2022-07-12


In [38]:
import json
import readline
from sutime import SUTime
from datetime import datetime

def checkDate(parsedData, referenceDate):
    flag = False
    for tag in parsedData:
        if tag['type'] == 'DATE':
            # if datetime.strptime(tag['value'], '%Y-%m-%d') <= datetime.strptime(referenceDate, '%Y-%m-%d'):
            try:
                if datetime.strptime(tag['value'], '%Y-%m-%d') <= datetime.strptime(referenceDate, '%Y-%m-%d'):
                    flag = True
            except:
                pass
    return flag

def extractDate(parsedData, referenceDate):
    focusTime = ''
    for tag in parsedData:
        if tag['type'] == 'DATE':
            try:
                if datetime.strptime(tag['value'], '%Y-%m-%d') <= datetime.strptime(referenceDate, '%Y-%m-%d'):
                    focusTime = tag['value']
                    return focusTime
            except:
                pass   
sutime = SUTime()
# print(json.dumps(sutime.parse(test_case), sort_keys=True, indent=4))
timeData = dict()
df_elements = df.sample(n=100)
# print(str)
count = 0
for data in df_elements.itertuples():
    headerParse = sutime.parse(data[2], reference_date=data[5])
    checkDate(headerParse, data[5])
    if not headerParse or not checkDate(headerParse, data[5]):
        summaryParse = sutime.parse(data[3], reference_date=data[5])
        checkDate(summaryParse, data[5])
        if not summaryParse or not checkDate(summaryParse, data[5]):
            details = data[4]
            lines = details.split('\n')
            del lines[-2]
            details = '\n'.join(lines)
            detailsParse = sutime.parse(details, reference_date=data[5])
            checkDate(detailsParse, data[5])
            if not detailsParse:
                count += 1
            else:
                timeData[data[0]] = dict()
                timeData[data[0]]["Creation_Date"] = data[5]
                timeData[data[0]]["focusTime"] = extractDate(detailsParse, data[5])
                timeData[data[0]]['Details'] = dict()
                timeData[data[0]]['Details']["Text"] = data[4]
                timeData[data[0]]['Details']["Tags"] = detailsParse
        else:
            timeData[data[0]] = dict()
            timeData[data[0]]["Creation_Date"] = data[5]
            timeData[data[0]]["focusTime"] = extractDate(summaryParse, data[5])
            timeData[data[0]]['Summary'] = dict()
            timeData[data[0]]['Summary']["Text"] = data[3]
            timeData[data[0]]['Summary']["Tags"] = summaryParse
    else:
        timeData[data[0]] = dict()
        timeData[data[0]]["Creation_Date"] = data[5]
        timeData[data[0]]["focusTime"] = extractDate(headerParse, data[5])
        timeData[data[0]]["Header"] = dict()
        timeData[data[0]]["Header"]["Text"] = data[2]
        timeData[data[0]]["Header"]["Tags"] = headerParse
    if timeData[data[0]]["focusTime"] == 'null':
        timeData[data[0]]["focusTime"] = data[5]
print ("Number of news with no time tag at all: ", count)
# # lines = df_elements['Detail'].head()
# with open('details.txt', 'w') as f:
#     f.writelines(lines)
# print(lines)
print(json.dumps(timeData, indent=4))

Number of news with no time tag at all:  0
{
    "6524": {
        "Creation_Date": "2022-09-07",
        "focusTime": "2022-09-06",
        "Details": {
            "Tags": [
                {
                    "timex-value": "2022-09-06",
                    "start": 335,
                    "end": 342,
                    "text": "Tuesday",
                    "type": "DATE",
                    "value": "2022-09-06"
                },
                {
                    "timex-value": "2022-09-06",
                    "start": 1796,
                    "end": 1803,
                    "text": "Tuesday",
                    "type": "DATE",
                    "value": "2022-09-06"
                }
            ]
        }
    },
    "6655": {
        "Creation_Date": "2022-09-12",
        "focusTime": "2022-09-09",
        "Details": {
            "Tags": [
                {
                    "timex-value": "2022-09-09",
                    "start": 501,
                    "e

In [29]:
# print(len(timeData[0]['Header']))
# for i in range(len(timeData)):
#     if timeData[i]['Header']:
#         print(timeData[i]['Header'])

import datetime
datetime.strptime('2022-08-01', '%Y-%m-%d') > datetime.strptime('2022-08-02', '%Y-%m-%d')

AttributeError: module 'datetime' has no attribute 'strptime'